## Please input your directory for the top level folder
folder name : SUBMISSION MODEL

In [1]:
dir_ = 'E:/Seminararbeit/Code/A1/' # input only here

#### setting other directory

In [2]:
raw_data_dir = dir_+'2. data/'
processed_data_dir = dir_+'2. data/processed/'
log_dir = dir_+'4. logs/'
model_dir = dir_+'5. models/'
submission_dir = dir_+'6. submissions/'

In [3]:
####################################################################################
########################### 1-1. recursive model by store ##########################
####################################################################################

In [4]:
ver, KKK = 'priv', 1
STORES_IDS = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']

In [5]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from multiprocessing import Pool

warnings.filterwarnings('ignore')

In [6]:
########################### Helpers
#################################################################################
## Seeder
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [7]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    df = df[df['d']>=START_TRAIN]
    
    df = df[df['store_id']==store]

    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2
    
    df = pd.concat([df, df3], axis=1)
    del df3
    
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    df = df.reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle(processed_data_dir+'test_'+store_id+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [8]:
########################### Model params
#################################################################################
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.015,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 3000,
#                     'n_estimators': 3,
                    'boost_from_average': False,
                    'verbose': 1,
                    'n_jobs': 1
} 


In [9]:
########################### Vars
#################################################################################
VER = 1                          
SEED = 42                        
seed_everything(SEED)            
lgb_params['seed'] = SEED        
N_CORES = psutil.cpu_count()     


#LIMITS and const
TARGET      = 'sales'            
START_TRAIN = 0                
END_TRAIN   = 1941 - 28*KKK      
P_HORIZON   = 28                 
USE_AUX     = False             

remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

ORIGINAL = raw_data_dir
BASE     = processed_data_dir+'processedgrid_part_1.pkl'
PRICE    = processed_data_dir+'processedgrid_part_2.pkl'
CALENDAR = processed_data_dir+'processedgrid_part_3.pkl'
LAGS     = processed_data_dir+'processedlags_df_28.pkl'
MEAN_ENC = processed_data_dir+'processedmean_encoding_df.pkl'


#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

In [10]:
_, MODEL_FEATURES = get_data_by_store(STORES_IDS[-1])
del _; gc.collect()

0

In [11]:
########################### Predict
#################################################################################

all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

main_time = time.time()

for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    grid_df = base_test.copy()
    
    # slow for loop version
    temp = []
    for a in ROLS_SPLIT:
        temp.append(make_lag_roll(a))
    temp = pd.concat(temp, axis=1)
    grid_df = pd.concat([grid_df, temp], axis=1)
    del temp; gc.collect()
    ###
    
    # fast multiprocessing version
    #     grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)
    ###
    
    for store_id in STORES_IDS:
        
        model_path = model_dir+'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 
        if USE_AUX:
            model_path = AUX_MODELS + model_path

        estimator = pickle.load(open(model_path, 'rb'))

        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id

        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])

    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))

    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds

Predict | Day: 1
##########  1.78 min round |  1.78 min total |  37732.01 day sales |
Predict | Day: 2
##########  1.59 min round |  3.36 min total |  35846.63 day sales |
Predict | Day: 3
##########  1.59 min round |  4.96 min total |  35231.96 day sales |
Predict | Day: 4
##########  1.59 min round |  6.55 min total |  35903.92 day sales |
Predict | Day: 5
##########  1.67 min round |  8.22 min total |  42927.37 day sales |
Predict | Day: 6
##########  1.60 min round |  9.82 min total |  52198.17 day sales |
Predict | Day: 7
##########  1.65 min round |  11.48 min total |  55278.53 day sales |
Predict | Day: 8
##########  1.72 min round |  13.19 min total |  44822.18 day sales |
Predict | Day: 9
##########  1.66 min round |  14.85 min total |  45421.76 day sales |
Predict | Day: 10
##########  1.62 min round |  16.47 min total |  39642.99 day sales |
Predict | Day: 11
##########  1.60 min round |  18.07 min total |  42019.93 day sales |
Predict | Day: 12
##########  1.79 min round | 

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.848261,0.795400,0.794611,0.838171,1.027779,1.222009,1.445239,0.945290,1.019310,...,0.899318,1.258442,1.210328,0.880430,0.795655,0.764226,0.871957,1.033397,1.154834,1.123887
1,HOBBIES_1_002_CA_1_evaluation,0.181783,0.167809,0.152716,0.166177,0.206811,0.293266,0.296793,0.238486,0.230887,...,0.258710,0.303964,0.333444,0.230255,0.214381,0.204716,0.209127,0.219027,0.301968,0.288280
2,HOBBIES_1_003_CA_1_evaluation,0.468899,0.408352,0.454861,0.440979,0.599572,0.827995,0.695588,0.535905,0.496406,...,0.628996,0.730210,0.750833,0.523109,0.461450,0.465226,0.508227,0.641242,0.724159,0.695044
3,HOBBIES_1_004_CA_1_evaluation,1.645641,1.399950,1.367059,1.559464,2.007527,2.677630,2.918485,1.697665,1.453143,...,1.869482,2.451186,2.968196,1.844546,1.565322,1.425019,1.354770,1.847923,2.843252,3.425445
4,HOBBIES_1_005_CA_1_evaluation,1.022801,0.883259,0.959263,1.021903,1.154769,1.366164,1.546872,1.065341,1.113875,...,1.106291,1.551614,1.643015,1.047924,0.911363,0.927251,0.966889,1.167693,1.545888,1.471905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,0.425706,0.406128,0.381083,0.372734,0.421276,0.466835,0.498523,0.508609,0.578160,...,0.510746,0.683349,0.879202,0.502777,0.426068,0.425239,0.401960,0.444662,0.478030,0.555873
30486,FOODS_3_824_WI_3_evaluation,0.318052,0.288158,0.285258,0.281319,0.303454,0.363643,0.350875,0.396717,0.435961,...,0.258869,0.369866,0.448530,0.285972,0.261097,0.239789,0.230912,0.233285,0.302802,0.308320
30487,FOODS_3_825_WI_3_evaluation,0.637820,0.517626,0.474612,0.494682,0.605469,0.782783,0.919022,0.930386,0.941985,...,0.955319,1.176954,1.484207,1.049948,0.715643,0.687567,0.545614,0.695499,0.861326,0.898185
30488,FOODS_3_826_WI_3_evaluation,0.943343,0.891239,0.752536,0.779710,0.886183,1.294287,1.320985,1.219451,1.185552,...,1.095082,1.476958,1.520480,1.027300,1.031962,0.895814,0.902419,1.152629,1.126893,1.353109


In [12]:
########################### Export
#################################################################################
submission = pd.read_csv(ORIGINAL+'sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv(submission_dir+'before_ensemble/submission_kaggle_recursive_store.csv', index=False)